# Zelros test technique : version simple
Import of necessary libraries

In [58]:
import tensorflow as tf
from tqdm._tqdm_notebook import tqdm_notebook 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import re

tqdm_notebook.pandas()
print(tf.__version__)

2.0.0


Read the dataset

In [67]:
train_df = pd.read_csv("../dataset/train.csv")

Remove ponctuations and unnecessary spaces in sentences as well as transfer to lowercase

In [68]:
def to_lower_case_and_rm_double_spaces_poncuation(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'\W',' ',sentence) # Remove ponctuation (non-word caracters)
    sentence = re.sub(r'\s+',' ',sentence) # Remove multiple space
    if sentence[-1]==" ": # Remove useless space at the end of the sentence
        sentence = sentence[:-1]
    return sentence

train_df["question_text"] = train_df["question_text"].progress_apply(to_lower_case_and_rm_double_spaces_poncuation)
print("Check the result on the first sentence : {}".format(train_df["question_text"][0]))


Check the result on the first sentence : how did quebec nationalists see their province as a nation in the 1960s


Split the dataset in train data and validation data

In [42]:
train_df, val_df = train_test_split(train_df, test_size=0.1)


Test the data repartition

In [43]:
percentage_in_train = train_df.groupby("target").count()["qid"][1]/train_df.size
percentage_in_val = val_df.groupby("target").count()["qid"][1]/val_df.size
print(f"Train dataset size: {train_df.size}, validation size: {val_df.size}, "
      f"{math.floor(val_df.size*100/train_df.size)}% of the training dataset size")
print("Percentage of positives in train = {:.2f} and in val {:.2f}".format(percentage_in_train,percentage_in_val))

Train dataset size: 3526527, validation size: 391839, 11% of the training dataset size
Percentage of positives in train = 0.02 and in val 0.02
